# SoH Estimation Experimentation for Ford

## Méthode de calcul du SoH

```
soh = battery_energy / (battery_level * net_capacity)
```

## Choix méthodologiques

### Filtrage par niveau de charge (SOC - State of Charge)
- **Critère** : Conservation uniquement des données avec `battery_level` entre **40% et 99%**
- **Justification** : Le SoH est plus fiable dans cette plage de charge. En dessous de 40%, les mesures peuvent être moins précises, et au-dessus de 99%, la charge complète peut introduire des biais.

### Inclusion des phases de charge et décharge
- **Choix** : Conservation des données en phase de charge ET de décharge
- **Justification** : Aucune corrélation significative n'a été observée entre le SoH et l'état de charge/décharge. Les deux types de phases apportent des informations valides.

### Agrégation par phase
- **Méthode choisie** : **Médiane du SoH par phase**
- **Justification** : La variance du SoH est plus faible avec la médiane qu'avec la somme d'énergie, ce qui indique une meilleure stabilité de la mesure.





In [ ]:
import numpy as np
import plotly.express as px

from core.pandas_utils import *
from core.s3.s3_utils import S3Service
from core.s3.settings import S3Settings
from core.spark_utils import create_spark_session
from core.sql_utils import *
from core.stats_utils import *
from transform.fleet_info.main import fleet_info

# Configuration
settings = S3Settings()
spark = create_spark_session(
    settings.S3_KEY,
    settings.S3_SECRET
)
s3 = S3Service()
company = "ford"

## 1. Extraction des données

In [ ]:
raw_tss = s3.read_parquet_df_spark(
    spark, "raw_ts/ford/time_series/raw_ts_spark.parquet"
).toPandas()


In [ ]:
# Compter le nombre de VIN uniques
nombre_vin_uniques = raw_tss["vin"].nunique()
print(f"Le nombre de VIN différents dans tss est : {nombre_vin_uniques}")

In [ ]:
# Récupération des informations des véhicules depuis la base de données
with get_connection() as con:
    cursor = con.cursor()
    cursor.execute("""
        SELECT vm.model_name, vm.type, vm.autonomy, v.vin, b.net_capacity 
        FROM vehicle v 
        LEFT JOIN vehicle_model vm ON v.vehicle_model_id = vm.id
        LEFT JOIN battery b ON b.id = vm.battery_id
    """)
    dbeaver_df = cursor.fetchall()
    dbeaver_df = pd.DataFrame(
        dbeaver_df, 
        columns=[desc[0] for desc in cursor.description]
    )


In [ ]:
# Fusion des données de véhicules avec les time series
raw_tss = raw_tss.merge(dbeaver_df, on="vin", how="left")

In [ ]:
raw_tss.columns

In [ ]:
types = {
    "vin": str,
    "date": "datetime64[ns]",
    "battery_energy": float,
    "battery_level": float,
    "odometer": float,
    "time_to_complete_charge": str,
    "charger_voltage": float,
    "status": str,
    "net_capacity": float,
    "model_name": str,
    "type": str,
    "autonomy": float,
    "battery_performance_status": str,
}
raw_tss = raw_tss.astype(types)



In [ ]:
def detect_charging(df, level_col="battery_level", ts_col="date", min_consecutive=2):
    df = df.sort_values(ts_col).reset_index(drop=True)
    df["delta"] = df[level_col].diff().fillna(0)

    charging = False
    states = []
    pos_count, neg_count = 0, 0

    for d in df["delta"]:
        if d > 0:
            pos_count += 1
            neg_count = 0
        elif d < 0:
            neg_count += 1
            pos_count = 0
        else:
            pass
        if pos_count >= min_consecutive:
            charging = True
        elif neg_count >= min_consecutive:
            charging = False

        states.append(charging)

    df["charging"] = states
    return df

def add_phase_id(df):
    df_copy = df.copy()
    changes = df_copy['charging'] != df_copy['charging'].shift(1)
    df_copy['phase_id'] = changes.cumsum()
    df_copy["phase_id"] = df_copy["phase_id"].astype(str)
    return df_copy

raw_tss = raw_tss.groupby("vin", group_keys=False).apply(
    lambda g: detect_charging(g, level_col="battery_level", ts_col="date", min_consecutive=2)
)
raw_tss = add_phase_id(raw_tss)

## 2. Analyse exploratoire des séries temporelles

In [ ]:
most_common_vin = raw_tss.groupby("vin").size().sort_values(ascending=False).idxmax()
print(f"VIN sélectionné : {most_common_vin}")
ts = raw_tss.query(f"vin == '{most_common_vin}'")

In [ ]:
px.scatter(ts, x="date", y="battery_energy", title=f"{most_common_vin}")

In [ ]:
px.scatter(ts, x="date", y="battery_level", title=f"{most_common_vin}", color="charging")

### 2.1. Étude de la corrélation pour battery_energy

In [ ]:
corr  = raw_tss.corr(numeric_only=True)
selected_column = "battery_energy"
selected_corr = corr[[selected_column]].sort_values(by=selected_column, ascending=False)

# heat map of the correlation matrix
px.imshow(selected_corr, title=f"Correlation Matrix for {selected_column}")


## 3. Filtrage initial des données

In [ ]:

ts = ts.query("odometer != 0")
tss = raw_tss.query("odometer != 0")
print(f"Nombre d'enregistrements après filtrage : {len(tss)}")

## 4. Calcul du SoH et réduction des dépendances


In [ ]:
tss["soh"] = tss["battery_energy"] / tss["battery_level"] / tss["net_capacity"]
ts["soh"] = ts["battery_energy"] / ts["battery_level"] / ts["net_capacity"]


### 4.1. Dépendance au niveau de charge (SOC)

**Observation** : Le SoH est plus fiable pour des niveaux de charge entre 40% et 99%.  
On filtre donc les données pour ne garder que ces valeurs.

In [ ]:
# Visualisation de la relation entre niveau de batterie et énergie
px.scatter(
    tss, 
    x="battery_level", 
    y="battery_energy", 
    color="net_capacity",
    title="Énergie de batterie vs Niveau de batterie"
)


In [ ]:

fig = px.scatter(
    tss,
    x="battery_level",
    y="soh",
   # color="net_capacity",
    height=600,
    title="État de santé (SoH) vs Niveau de batterie",
    trendline="ols",
    trendline_scope="overall",
    hover_data=["vin"]
)
fig.show()

In [ ]:
ts_soc_filtered = ts.query("battery_level > 0.4").query("battery_level < 0.99").copy()
tss_soc_filtered = tss.query("battery_level > 0.4").query("battery_level < 0.99").copy()

In [ ]:
# Calcul de la matrice de corrélation
corr =(tss_soc_filtered[tss_soc_filtered["net_capacity"] == 72.6][['battery_energy', 'battery_level',"soh"]].corr(numeric_only=True))
selected_column = "soh"
selected_corr = corr[[selected_column]].sort_values(by=selected_column, ascending=False)

# Visualisation de la matrice de corrélation
px.imshow(
    selected_corr, 
    title=f"Matrice de corrélation pour {selected_column}",
    labels=dict(x="Variables", y="Corrélation", color="Coefficient")
)

### 4.2. Dépendance à l'état de charge/décharge

**Observation** : Il n'y'a pas de corrélation entre les phases de charges/décharges on peut donc garder les deux.


In [ ]:
# Visualisation du SoH dans le temps 
px.scatter(
    ts_soc_filtered,
    x="date",
    y="soh",
    color="charging",
    title="SoH dans le temps (charge vs décharge)"
)

In [ ]:
# Calcul de la matrice de corrélation
corr =(tss_soc_filtered[['battery_energy', 'battery_level',"soh", "charging"]].corr(numeric_only=True))
selected_column = "soh"
selected_corr = corr[[selected_column]].sort_values(by=selected_column, ascending=False)

# Visualisation de la matrice de corrélation
px.imshow(
    selected_corr, 
    title=f"Matrice de corrélation pour {selected_column}",
    labels=dict(x="Variables", y="Corrélation", color="Coefficient")
)

## 5. Calcul du SoH final

### 5.1. Groupement et agrégation du SoH

**Observation**: la variance est plus élevé entre pour la somme à la charge qu'en prenant la médiane.    
**Conclusion**: On choisit donc la médiane par phase.

#### 5.1.1. Méthode 1 : Médiane par phase

In [ ]:
# Agrégation par phase : médiane du SoH
soh_median = tss_soc_filtered.groupby(["vin", "phase_id"], as_index=False).agg({
    "soh": "median",
    "odometer": "max",
    "date": "max",
})

In [ ]:
soh_median_var = soh_median.groupby("vin").agg(
    soh_median = ("soh", "median"),
    soh_mean = ("soh", "mean"),
    soh_std = ("soh", "std"),
    number_week = ("soh", "count"),
    min = ("soh", "min"),
    max = ("soh", "max"),
)
px.scatter(soh_median_var, x="number_week", y="soh_std", )

#### 5.1.2. Méthode 2 : Somme de l'énergie de la batterie par phase

In [ ]:
raw_tss["expected_battery_energy"] = (
    raw_tss["net_capacity"] * raw_tss["battery_level"] / 100
)

In [ ]:
tss_energy = raw_tss.groupby(["vin", "phase_id"], as_index=False).agg({
    "battery_energy": "sum",
    "expected_battery_energy": "sum",
    "odometer": "max",
    "date": "max",
})

In [ ]:
# Calcul du SoH basé sur la somme d'énergie
tss_energy["soh"] = (
    tss_energy["battery_energy"] / tss_energy["expected_battery_energy"] / 100
)


In [ ]:
soh_ener_var = tss_energy.groupby("vin").agg(
    soh_median = ("soh", "median"),
    soh_mean = ("soh", "mean"),
    soh_std = ("soh", "std"),
    number_week = ("soh", "count"),
    min = ("soh", "min"),
    max = ("soh", "max"),
)
px.scatter(soh_ener_var, x="number_week", y="soh_std", )

### 5.2 Agregation week

**Observation** : Peut de différence entre la moyenne et la médiane.

#### 5.2.1 mediane par semaine


In [ ]:
# Fréquence de mise à jour : agrégation par semaine
UPDATE_FREQUENCY = pd.Timedelta(days=7)


In [ ]:
# Création de la colonne WEEK pour l'agrégation hebdomadaire
soh_median["WEEK"] = (
    pd.to_datetime(soh_median["date"], format="mixed")
    .dt.floor(UPDATE_FREQUENCY)
    .dt.tz_localize(None)
    .dt.date.astype("datetime64[ns]")
)

In [ ]:
# Agrégation hebdomadaire : médiane du SoH par VIN et par semaine
soh_week_median = soh_median.groupby(["vin", "WEEK"], as_index=False).agg({
    "soh": "median",
    "odometer": "max",
})



In [ ]:
soh_var_median = soh_week_median.groupby("vin").agg(
    soh_median = ("soh", "median"),
    soh_mean = ("soh", "mean"),
    soh_std = ("soh", "std"),
    number_week = ("soh", "count"),
    min = ("soh", "min"),
    max = ("soh", "max"),
).eval("soh_diff=max-min")

px.scatter(soh_var_median, x="number_week", y="soh_diff", )

In [ ]:
# Visualisation de l'évolution du SoH dans le temps (méthode médiane par phase)
px.scatter(
    soh_week_median, 
    x="WEEK", 
    y="soh", 
    color="vin",
    title="Évolution du SoH dans le temps (méthode médiane par phase)",
    labels={
        "WEEK": "Semaine",
        "soh": "SoH"
    }
)

#### 5.2.3 Moyenne pondéré par odomètre

In [ ]:
soh_median_clean = soh_median.dropna(subset=["odometer", 'soh']).copy()
d = {}
for vin in soh_median_clean["vin"].unique():
    d[vin] = {}
    df_temp = soh_median_clean[soh_median_clean["vin"] == vin]
    for week in df_temp["WEEK"].unique():
        df_temp_week = df_temp[df_temp["WEEK"] == week]
        df_temp_week["odometer_diff"] = df_temp_week["odometer"].diff().shift(-1)
        soh = weighted_mean(df_temp_week["soh"], df_temp_week["odometer_diff"])
        d[vin][week] = soh


In [ ]:
pd.DataFrame(d).T

In [ ]:
from core.stats_utils import weighted_mean

# Préparation des données
soh_median_clean = soh_median.dropna(subset=["odometer", 'soh']).copy()

# Fonction d'agrégation pour calculer la moyenne pondérée
def weighted_soh_agg(group):
    """Calcule la moyenne pondérée du SoH pour un groupe"""
    return pd.Series({
        "soh": weighted_mean(group["soh"], group["odometer"]),
        "odometer": group["odometer"].max()
    })

# Agrégation hebdomadaire avec moyenne pondérée par odomètre
soh_week_weighted = soh_median_clean.groupby(["vin", "WEEK"], as_index=False, group_keys=False).apply(
    weighted_soh_agg
)


#### 5.2.2 Moyenne par semaine 

In [ ]:
# Fréquence de mise à jour : agrégation par semaine
UPDATE_FREQUENCY = pd.Timedelta(days=7)


In [ ]:
# Création de la colonne WEEK pour l'agrégation hebdomadaire
soh_median["WEEK"] = (
    pd.to_datetime(soh_median["date"], format="mixed")
    .dt.floor(UPDATE_FREQUENCY)
    .dt.tz_localize(None)
    .dt.date.astype("datetime64[ns]")
)

In [ ]:
# Agrégation hebdomadaire : médiane du SoH par VIN et par semaine
soh_week_mean = soh_median.groupby(["vin", "WEEK"], as_index=False).agg({
    "soh": "mean",
    "odometer": "max",
})



In [ ]:
soh_var_mean = soh_week_mean.groupby("vin").agg(
    soh_median = ("soh", "median"),
    soh_mean = ("soh", "mean"),
    soh_std = ("soh", "std"),
    number_week = ("soh", "count"),
    min = ("soh", "min"),
    max = ("soh", "max"),
).eval("soh_diff=max-min")
px.scatter(soh_var_mean, x="number_week", y="soh_diff", )

In [ ]:
# Visualisation de l'évolution du SoH dans le temps (méthode médiane par phase)
px.scatter(
    soh_week_mean, 
    x="odometer", 
    y="soh", 
    color="vin",
    title="Évolution du SoH dans le temps (méthode médiane par phase)",
    labels={
        "WEEK": "Semaine",
        "soh": "SoH"
    }
)